# import lib

In [ ]:
import pandas as pd
import pyodbc

# load csv file and create dataframe

In [ ]:

csv_file_path1 = r"D:\Workplace\code\ADY_prj\Used cars prediction\dataset\bonbanh_used_cars_details.csv"
csv_file_path2 = r"D:\Workplace\code\ADY_prj\Used cars prediction\dataset\new_cars_full_data.csv"
csv_file_path3 = r"D:\Workplace\code\ADY_prj\Used cars prediction\dataset\Used_car_data_promax.csv"

df1 = pd.read_csv(csv_file_path1)
df2 = pd.read_csv(csv_file_path2)
df3 = pd.read_csv(csv_file_path3)

In [ ]:
df1_renamed = df1.rename(columns={
    'Body Type': 'BodyType',
    'Engine Type': 'EngineType',
    'Color inside': 'ColorInside'
})

df3_renamed = df3.rename(columns={
    'Body Type': 'Body_Type',
    'Fuel Type': 'Fuel_Type',
    'Sale Date': 'sale_date'
})

# handle missing values

In [ ]:

dfh1 = df1_renamed.fillna(value='None')
dfh2 = df2.fillna(value='None')
dfh3 = df3_renamed.fillna(value='None')

# convert all value type to 'string'

In [ ]:

dfh1 = dfh1.astype(str)
dfh2 = dfh2.astype(str)
dfh3 = dfh3.astype(str)


# sql sever connection setup

In [ ]:
server = 'NITRO\\SQLEXPRESS' 
database = 'cars_data' 

driver = 'ODBC Driver 17 for SQL Server'

# Adjust connection string for Windows Authentication
connection_string = f'DRIVER={{{driver}}};SERVER={server};DATABASE={database};Trusted_Connection=yes;'
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

# create table in sql sever

In [ ]:
# create table bonbanh
create_table_query = '''
IF NOT EXISTS (SELECT * FROM sys.objects WHERE object_id = OBJECT_ID(N'bonbanh_inf') AND type in (N'U'))
BEGIN
    CREATE TABLE bonbanh_inf (
        name VARCHAR(255),
        Price VARCHAR(255),
        SellDate VARCHAR(255),
        Date VARCHAR(255),
        Condition VARCHAR(255),
        Mileage VARCHAR(255),
        Origin VARCHAR(255),
        BodyType VARCHAR(255),
        EngineType VARCHAR(255),
        Color VARCHAR(255),
        ColorInside VARCHAR(255),
        Seats VARCHAR(255),
        Doors VARCHAR(255)
    );
END;
'''

try:
    cursor.execute(create_table_query)
    conn.commit()
    print("Table 'bonbanh_inf' created or already exists.")
except Exception as e:
    print(f"Error creating general table: {e}")


# create table newcar
create_table_query = '''
IF NOT EXISTS (SELECT * FROM sys.objects WHERE object_id = OBJECT_ID(N'newcar_inf') AND type in (N'U'))
BEGIN
    CREATE TABLE newcar_inf (
        name VARCHAR(255),
        link VARCHAR(255),
        manufacturer VARCHAR(255),
        origin VARCHAR(255),
        body_type VARCHAR(255),
        seating_capacity VARCHAR(255),
        engine VARCHAR(255),
        torque VARCHAR(255),
        transmission VARCHAR(255),
        max_power VARCHAR(255),
        drive VARCHAR(255),
        fuel_type VARCHAR(255),
        fuel_capacity VARCHAR(255)
    );
END;
'''

try:
    cursor.execute(create_table_query)
    conn.commit()
    print("Table 'newcar_inf' created or already exists.")
except Exception as e:
    print(f"Error creating general table: {e}")



# Correct table creation for 'usedCar'
create_table = '''
    IF NOT EXISTS (SELECT * FROM sys.objects WHERE object_id = OBJECT_ID(N'usedCar') AND type in (N'U'))
    BEGIN
        CREATE TABLE usedCar (
            car_id INT,
            title VARCHAR(255),
            year VARCHAR(255),
            Body_Type VARCHAR(50), 
            Origin VARCHAR(50),
            Mileage VARCHAR(255),
            City VARCHAR(50),
            District VARCHAR(50),
            Transmission VARCHAR(50),
            Fuel_Type VARCHAR(50),
            Price VARCHAR(255),
            sale_date VARCHAR(50),
            Manufacturer VARCHAR(255)
        );
    END;
'''

try:
    cursor.execute(create_table)
    conn.commit()
    print("Table 'usedCar' created or already exists.")
except Exception as e:
    print(f"Error creating 'usedCar' table: {e}")

# insert data from data frame into sql sever

In [ ]:
# Insert data into 'bonbanh_inf'
for index, row in dfh1.iterrows():
    insert_bonbanh_query = '''
    INSERT INTO bonbanh_inf (name, Price, SellDate, Date, Condition, Mileage, Origin, BodyType, EngineType, Color, ColorInside, Seats, Doors)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?);
    '''
    try:
        cursor.execute(insert_bonbanh_query, 
                       row['Name'], 
                       row['Price'], 
                       row['SellDate'], 
                       row['Date'], 
                       row['Condition'], 
                       row['Mileage'], 
                       row['Origin'], 
                       row['BodyType'], 
                       row['EngineType'], 
                       row['Color'], 
                       row['ColorInside'], 
                       row['Seats'], 
                       row['Doors'])
        conn.commit()
    except Exception as e:
        print(f"Error inserting into 'bonbanh_inf' for row {index}: {e}")

# Insert data into 'newcar_inf'
for index, row in dfh2.iterrows():
    insert_newcar_query = '''
    INSERT INTO newcar_inf (name, link, manufacturer, origin, body_type, seating_capacity, engine, torque, transmission, max_power, drive, fuel_type, fuel_capacity)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?);
    '''
    try:
        cursor.execute(insert_newcar_query, 
                       row['name'], 
                       row['link'], 
                       row['manufacturer'], 
                       row['origin'], 
                       row['body_type'], 
                       row['seating_capacity'], 
                       row['engine'], 
                       row['torque'], 
                       row['transmission'], 
                       row['max_power'], 
                       row['drive'], 
                       row['fuel_type'], 
                       row['fuel_capacity'])
        conn.commit()
    except Exception as e:
        print(f"Error inserting into 'newcar_inf' for row {index}: {e}")

# Insert data into 'usedCar'
for index, row in dfh3.iterrows():
    insert_usedcar_query = '''
    INSERT INTO usedCar (car_id, title, year, Body_Type, Origin, Mileage, City, District, Transmission, Fuel_Type, Price, sale_date, Manufacturer)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?);
    '''
    try:
        cursor.execute(insert_usedcar_query, 
                       row['car_id'], 
                       row['Title'], 
                       row['Year'], 
                       row['Body_Type'], 
                       row['Origin'], 
                       row['Mileage'], 
                       row['City'], 
                       row['District'], 
                       row['Transmission'], 
                       row['Fuel_Type'], 
                       row['Price'], 
                       row['sale_date'], 
                       row['manufacturer'])
        conn.commit()
    except Exception as e:
        print(f"Error inserting into 'usedCar' for row {index}: {e}")

# close connection

In [ ]:
# Close connection
cursor.close()
conn.close()

print("Data successfully inserted into SQL Server.")